<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA-25/blob/main/lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***


---



# Занятие №3
# Поиск аномалий методами машинного обучения

**5 марта 2025г.**

---

С точки зрения машинного оучения задачу поиска поиска аномалий можно разделить на два возможных типа:
* **Outlier detection** (поиск выбросов): в тренировочной выборке содержатся выбросы, которые определяются как наблюдения, лежащие далеко от остальных. Таким образом, алгоритмы для детектирования выбросов пытаются найти регионы, где сосредоточена основная масса тренировочных данных, игрорируя аномальные наблюдения.
* **Novelty detection** (поиск "новизны"): тренировочная выборка не загрязнена выбросами, и мы хотим научиться отвечать на вопрос "является ли новое наблюдение выбросом".

In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import seaborn as sns
sns.set_theme(style="whitegrid", rc={'figure.figsize':(15,6)})

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM

from tqdm import tqdm
import json

import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go

pd.options.mode.chained_assignment = None

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

## Загрузка данных

In [ ]:
data = pd.read_parquet("./data.gzip")

# signals = [
#     'GTA1.DBinPU.Aldi', 'GTA1.DBinPU.Alvna', 'GTA1.DBinPU.Alzzo',
#     'GTA1.DBinPU.Bo', 'GTA1.DBinPU.fi', 'GTA1.DBinPU.nst',
#     'GTA1.DBinPU.ntk', 'GTA1.DBinPU.P', 'GTA1.DBinPU.Pk', 'GTA1.DBinPU.Pvh',
#     'GTA1.DBinPU.Qtg', 'GTA1.DBinPU.Tk', 'GTA1.DBinPU.Tn', 'GTA1.DBinPU.Tt']

# data = data.loc[:, signals]

In [ ]:
data.head()

Чтение файла с описанием сигналов

In [ ]:
with open(f'./option_0/description.json', 'r', encoding = "utf-8") as f:
    description = json.load(f)

Составим словарь для трактовки наименований сигналов

In [ ]:
kks_to_description = {param['real_kks']: f"{param['description']}, [{param['unit']}]"
for param in description if param['real_kks'] in data.columns}

description_to_kks = { f"{param['description']}, [{param['unit']}]": param['real_kks']
for param in description if param['real_kks'] in data.columns}

### Деление на тренировочную и тестовую выборки

Разделение данных на **тренировочную** и **тестовую** выборки необходимо для оценки качества модели машинного обучения и предотвращения переобучения.  

**Основные цели разделения:**
1. **Обучение модели** – тренировочная выборка (training set) используется для подбора параметров модели, на основе этих данных модель «учится» находить закономерности.  
2. **Оценка обобщающей способности** – тестовая выборка (test set) служит для проверки, насколько хорошо модель работает на новых, невидимых данных ( как модель справляется с реальными задачами, которых она не видела во время обучения).  

**Почему нельзя тестировать модель на тех же данных, на которых она обучалась?**
Если модель тестировать на тех же данных, на которых она училась, она может просто «запомнить» их вместо того, чтобы действительно выявлять зависимости. Это приведёт к **переобучению** (overfitting), когда модель хорошо работает на обучающих данных, но плохо справляется с новыми примерами.  

Обычно данные делят следующим образом:
- **Тренировочная выборка** – 70-80%  
- **Тестовая выборка** – 20-30%  

Дополнительно иногда используют **валидационную выборку** (validation set) для подбора гиперпараметров модели.

Сформируем тренировочную и тестовую выборки.

In [ ]:
shuffled_data = data.sample(frac=1)

data_train = shuffled_data.iloc[:round(shuffled_data.shape[0]*0.8), :]
data_test = shuffled_data.iloc[round(shuffled_data.shape[0]*0.8):, :]

Добавим шум на половину тестовых данных и отметим их как аномальные

In [ ]:
def generate_binary_matrix(rows, cols, prob_ones=0.3):
    matrix = np.zeros((rows, cols), dtype=int)  # Создаем матрицу из нулей

    for i in range(rows):
        # Генерируем случайные 0 и 1 с заданной вероятностью (без гарантированной 1)
        row = np.random.choice([0, 1], size=cols, p=[1 - prob_ones, prob_ones])

        # Если в строке нет 1, вставляем её в случайное место
        if not np.any(row):
            row[np.random.randint(0, cols)] = 1

        matrix[i] = row  # Записываем строку в матрицу

    return matrix

In [ ]:
NUMBER_OF_ANOMALY_POINTS = round(data_test.shape[0]*0.5)

data_test["anomaly"] = 0
data_test.loc[
    data_test.iloc[:NUMBER_OF_ANOMALY_POINTS].index, ["anomaly"]] = 1

mask = generate_binary_matrix(NUMBER_OF_ANOMALY_POINTS, data.shape[1], prob_ones=0.2)

bias = mask * np.random.choice(
    np.linspace(-0.05,0.05,num=10),
    size=[NUMBER_OF_ANOMALY_POINTS, data.shape[1]],
    p=np.full(10,0.1))

data_test.loc[
    data_test.iloc[:NUMBER_OF_ANOMALY_POINTS].index,
    data_test.columns != "anomaly"] = data_test[data_test["anomaly"] == 1].loc[:,data_test.columns != "anomaly"] * (1 + bias)

In [ ]:
data_test.columns

In [ ]:
fig = plt.figure();

KKS_FOR_PLOT = "GTA1.DBinPU.ntk"

data_test[data_test["anomaly"] == 0][KKS_FOR_PLOT].plot(style='.', label="Нормальные данные");
data_test[data_test["anomaly"] == 1][KKS_FOR_PLOT].plot(style='.', label="Зашумленные данные");
plt.title(kks_to_description [KKS_FOR_PLOT], fontsize=20);
plt.legend(prop={'size': 16});

# fig.savefig('fig9.png', dpi=fig.dpi, bbox_inches='tight');

### Нормализация и стандартизация данных

**Нормализация** и **стандартизация** данных – это методы предобработки, используемые для приведения данных к единому масштабу, что улучшает работу моделей машинного обучения.  

**Зачем это нужно?**
1. **Разные масштабы признаков**
   - **Электрическая мощность** может измеряться в **кВт** (тысячи).
   - **Температура газов за ГТУ** – в **°C** (сотни).
   - **Положение топливного клапана** – в процентах (0-100).
   Без приведения к единому масштабу модель может считать признаки с большими значениями более важными, что искажает результаты.

2. **Ускорение сходимости градиентных методов**
   Если признаки имеют разные диапазоны, градиентный спуск будет сходиться медленно или нестабильно.

3. **Устойчивость к численным проблемам**
   Некоторые алгоритмы (например, метод k-ближайших соседей, линейная регрессия, нейросети) чувствительны к масштабу данных.

---

**Разница между нормализацией и стандартизацией**

| Метод          | Как работает | Когда применять |
|---------------|-------------|----------------|
| **Нормализация (Min-Max Scaling)** | Приводит значения к диапазону [0,1] или [-1,1]:  $$ x' = (x - min) / (max - min) $$ | Когда важны **границы диапазона** (например, при работе с нейросетями). |
| **Стандартизация (Z-Score Scaling)** | Приводит к среднему 0 и стандартному отклонению 1: $$ x' = (x - \mu) / \sigma $$ | Когда данные имеют **гауссово распределение** или важны относительные отклонения. |

---

Стандартизацию стоит применять при использовании алгоритмов, которые основываются на измерении расстояний, например, k ближайших соседей или метод опорных векторов (SVM).

Для поиска аномалий будем использовать модель на основе метода опорных векторов, поэтому выполним стандартизацию данных на которых будем обучать модель.

In [ ]:
# scaler = preprocessing.MinMaxScaler() # нормализация даннх
scaler = preprocessing.StandardScaler() # стандартизация данных

X_train = pd.DataFrame(
    scaler.fit_transform(data_train),
    columns=data_train.columns,
    index=data_train.index)

X_test = pd.DataFrame(
    scaler.transform(data_test[data.columns]),
    columns=data.columns,
    index=data_test.index)

X_train.describe()

In [ ]:
# X_train = X_train.iloc[:round(0.1*X_train.shape[0]),:]
# X_test = X_test.iloc[:round(0.1*X_train.shape[0]),:]

In [ ]:
print(X_train.shape)
print(X_test.shape)

## Метод опорных векторов (Support Vector Machines — SVM)

Один из наиболее популярных методов обучения, который применяется для решения задач классификации и регрессии. Основная идея метода заключается в построении гиперплоскости, разделяющей объекты выборки оптимальным способом. Алгоритм работает в предположении, что чем больше расстояние (зазор) между разделяющей гиперплоскостью и объектами разделяемых классов, тем меньше будет средняя ошибка классификатора.

![svm image](https://frankworkshophome.wordpress.com/wp-content/uploads/2019/12/7-1.png)

Класс OneClassSVM реализует одноклассную SVM, которая используется для обнаружения выбросов. Если мы имеем дело с задачей novelty detection, где для тренировки нам доступны только "хорошие" наблюдения без аномалий, то мы можем воспользоваться этой моделью и научиться для каждого нового наблюдения говорить, является ли оно аномальным или нет.

Общая идея OneClassSVM преобразовать признаковое пространство и провести разделяющую гиперплоскость так, чтобы наблюдения лежали как можно дальше от начала координат.
В результате мы получаем границу, по одну сторону которой максимально плотно упакованы наблюдения из нашей чистой тренировочной выборки, а по другую будут находится аномальные значения, не похожие на то, что алгоритм видел во время обучения.

**Плюсы и минусы**  
\+ Благодаря kernel trick, модель способна проводить нелинейные разделяющие границы  
\+ Особенно удобно использовать, когда в данных недостаточно "плохих" наблюдений, чтобы использовать стандартный подход обучения с учителем — бинарную классификацию  
\- Может очень сильно переобучиться и выдавать большое количество ложно отрицательных результатов, если разделяющий зазор слишком мал  
\- И, конечно, нужно быть абсолютно уверенным, что тренировочные данные не содержат никаких выбросов, иначе алгоритм будет считать их нормальными наблюдениями

In [ ]:
kernel = widgets.Dropdown(
    options=['linear', 'poly', 'rbf', 'sigmoid'],
    value='rbf',
    description='Kernel:',
    disabled=False,
)

nu = widgets.FloatSlider(
    value=0.1,
    min=0,
    max=1,
    step=0.01,
    description='nu:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

widgets.VBox([
    widgets.Label('Тип ядра, который будет использоваться в алгоритме'),
    kernel,
    widgets.Label('Верхняя граница доли ошибок обучения и нижняя граница доли опорных векторов'),
    nu])

---

One-Class SVM ищет границу, которая отделяет "нормальные" объекты от выбросов (аномалий). Ядро влияет на форму этой границы.

- **`kernel='rbf'`** (радиальная функция) — самый универсальный вариант, **подходит для большинства задач**.
- **Другие ядра (`linear`, `poly`, `sigmoid`)** могут работать лучше в специфических случаях, но требуют подбора параметров.

---

`nu` – это **гиперпараметр, управляющий чувствительностью модели** к аномалиям. Он задает **верхнюю границу доли выбросов** (аномалий) и **нижнюю границу доли опорных векторов** (управляет балансом между нормальными данными и выбросами).

- `nu` принимает значения от **0 до 1**.
- Чем **меньше `nu`**, тем **меньше** точек модель считает аномалиями (детекции только редких экстремальных выбросов).
- Чем **больше `nu`**, тем **больше** точек будут признаны выбросами (обнаружение даже слабые аномалии).
- **Типичное значение:** `nu=0.01` (1% выбросов) или `nu=0.1` (10% выбросов).


In [ ]:
ocsvm = OneClassSVM(kernel=kernel.value, nu=nu.value)
ocsvm.fit(X_train)

print(f"Число опорных векторов - {ocsvm.n_support_}")

Оценим тестовые данные

In [ ]:
data_test.loc[:,'anomaly_svm'] = (ocsvm.predict(X_test) == -1).astype(int)

### Визуализация аномальных данных

In [ ]:
fig = plt.figure();

signal = 'GTA1.DBinPU.P'

plt.scatter(
    data_test[data_test['anomaly'] == 1][signal].index,
    data_test[data_test['anomaly'] == 1][signal].values,
    s=70, facecolors='none', edgecolors='r', label= "Аномальные данные");

plt.scatter(
    data_test[data_test['anomaly_svm'] == 1][signal].index,
    data_test[data_test['anomaly_svm'] == 1][signal].values,
    s=10, facecolors='r', edgecolors='r', label= "Выявлена аномалия");

plt.scatter(
    data_test[data_test['anomaly'] == 0][signal].index,
    data_test[data_test['anomaly'] == 0][signal].values,
    s=70, facecolors='none', edgecolors='g', label= "Нормальные данные");

plt.scatter(
    data_test[data_test['anomaly_svm'] == 0][signal].index,
    data_test[data_test['anomaly_svm'] == 0][signal].values,
    s=10, facecolors='g', edgecolors='g', label= "Не выявлена аномалия");

plt.title(kks_to_description [signal], fontsize=20);
plt.legend(prop={'size': 16});

# fig.savefig('fig10.png', dpi=fig.dpi, bbox_inches='tight');

In [ ]:
try:
    scat_1 = data_test.groupby('anomaly_svm').get_group(1)
    scat_0 = data_test.groupby('anomaly_svm').get_group(0)
except KeyError:
    print("Не удалось разделить данные")

params_dropdown = widgets.Dropdown(
    options=data_test.columns,
    description='Параметр:',
    disabled=False,
    value=None
)

out = widgets.Output()
display(out)

with out:
    display(params_dropdown)

@out.capture()
def params_dropdown_eventhandler(change):

    clear_output()
    display(params_dropdown)
    # selected_param = selected_params_kks[list(selected_params_description).index(change.new)]


    fig, axes = plt.subplots(1, 1, figsize=(15,5))
    plt.title(f"{change.new} - {kks_to_description [change.new]}")
    plt.plot(scat_1[change.new], 'r.', markersize=5, label='Аномалии')
    plt.plot(scat_0[change.new], 'g.', markersize=5, label='Норма')
    plt.legend()
    display(fig)

params_dropdown.observe(params_dropdown_eventhandler, names='value')

### Оценка модели

`confusion_matrix` — это **матрица ошибок (матрица конфузии)**, которая показывает, насколько точно модель классифицирует объекты разных классов. Она помогает оценить качество классификации.  

**Разбор компонентов:**
- **TP (True Positive)** – модель правильно предсказала `1` (истинные положительные).  
- **TN (True Negative)** – модель правильно предсказала `0` (истинные отрицательные).  
- **FP (False Positive)** – модель ошибочно предсказала `1`, но был `0` (ложноположительные).  
- **FN (False Negative)** – модель ошибочно предсказала `0`, но был `1` (ложноотрицательные).  

---

**Если много FP (ложноположительных)** → модель часто ошибочно классифицирует объекты как положительные → **плохая специфичность**.  
**Если много FN (ложноотрицательных)** → модель пропускает настоящие положительные случаи → **низкая чувствительность**.  
**Баланс между FP и FN важен в разных задачах** (например, в медицине лучше избегать FN, а в спаме – FP).  


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

conf_mat = confusion_matrix(data_test['anomaly'], data_test['anomaly_svm'])
ConfusionMatrixDisplay(conf_mat).plot()
plt.show()

In [ ]:
tn, fp, fn, tp = conf_mat.ravel()
print("True Negative:", tn)
print("False Negative:", fn)
print("True Positive:", tp)
print("False Positive:", fp)

При оценке моделей машинного обучения важно понимать **accuracy (точность), precision (прецизионность), recall (полнота)** и **F1-score**. Они основаны на **confusion matrix** и помогают анализировать качество классификации.  

---

**1. Accuracy (доля правильных ответов)**  
**Accuracy** — это **общая точность модели**, доля **правильно предсказанных объектов** среди всех (пропорция верно предсказанных наблюдений):  

$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

Информативна если классы **сбалансированы** (примерно одинаковое количество примеров каждого класса).  
**Проблема:** Если классы **несбалансированы** (например, 95% класса 0 и 5% класса 1), модель может просто всегда предсказывать 0 и получить **95% accuracy, но это плохая модель!**  

---

**2. Precision (прецизионность, точность для положительного класса)**  
**Precision** показывает, насколько предсказания **"положительный" (1)** действительно правильные (с какой вероятностью положительные предсказания правильные):  

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

Полезна если важно **избежать ложных тревог (FP)**.  
Пример: В **медицине (рак, COVID)**, если мы говорим пациенту "у вас болезнь", но он здоров (**FP**), это может быть катастрофично.  

---

**3. Recall (полнота, чувствительность, чувствительность для положительного класса)**  
**Recall** показывает, сколько **реальных положительных случаев модель правильно нашла**:  

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

Полезна если важно **не пропустить настоящие случаи (FN)**.  
Пример: В **распознавании мошенничества** важно обнаружить **все мошеннические операции** (даже если появятся FP).  

---

**4. F1-score (среднее Precision и Recall)**  
**F1-score** — это **среднее гармоническое precision и recall**, дающее баланс между FP и FN:  

$$
\text{F1} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

Если **классы несбалансированы**, и нужно одновременно учитывать precision и recall.  
Если важно **не только избежать ложных тревог, но и не пропустить важные случаи**.  

---

- **Accuracy** хороша при **равномерном распределении классов**.  
- **Precision** и **recall** важны в **несбалансированных данных**.  
- **F1-score** балансирует между precision и recall.  

Чтобы получить полное понимание качества модели следует использовать **несколько метрик вместе**!

***accuracy*** - пропорция верно предсказанных наблюдений  
***precision*** - с какой вероятностью положительные предсказания правильные  
***recall*** - отражает способность модели определять наблюдения положительного класса  
***F1*** - какое количество названных положительными наблюдений являются истинноположительными:

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print(f"accuracy - {accuracy_score(data_test['anomaly'], data_test['anomaly_svm'])*100:0.2f}%")
print(f"precision - {precision_score(data_test['anomaly'], data_test['anomaly_svm'], average='binary')*100:0.2f}%")
print(f"recall - {recall_score(data_test['anomaly'], data_test['anomaly_svm'], average='binary')*100:0.2f}%")
print(f"F1 - {f1_score(data_test['anomaly'], data_test['anomaly_svm'], average='binary')*100:0.2f}%")

Значения метрик **зависят от конкретной задачи** и требований бизнеса. Однако есть **общие рекомендации**:

| Метрика  | Хорошее значение  | Плохое значение |
|----------|------------------|----------------|
| **Accuracy**  | **> 90%** (если классы сбалансированы) | < 50% (хуже случайного угадывания) |
| **Precision** | **> 80%** (если важно минимизировать FP) | < 50% (слишком много ложных срабатываний) |
| **Recall**    | **> 80%** (если важно минимизировать FN) | < 50% (пропускаем слишком много важных случаев) |
| **F1-score**  | **> 70-80%** (если баланс precision/recall важен) | < 50% (модель плохо работает) |

---

Метод **`score_samples(X)`** в **One-Class SVM** вычисляет **оценку аномальности** (или аффинности) для каждого объекта в `X`.  
Чем **ниже** значение, тем **больше вероятность**, что объект **аномальный**.  

---

One-Class SVM использует **ядровое пространство** для разделения данных.  
Метод `score_samples(X)` возвращает **оценку расстояния** от точки `X` до **границы "нормальности"**, определенной моделью.  

**Чем ниже `score_samples`, тем объект дальше от "нормы".**  

---


**Как интерпретировать `score_samples`?**
- **Высокие значения (`>0`)** → объект похож на нормальные данные.  
- **Низкие значения (`<0`)** → объект аномальный.  
- **Граница аномалий находится около `decision_function(X) = 0`**.  

Если нужно определить **фактические аномалии**, следует использовать `decision_function(X)` или `predict(X)`.


---

| Метод | Что делает? | Когда использовать? |
|--------|------------|---------------------|
| **`score_samples(X)`** | Возвращает **сырую оценку аномальности** (может быть больше или меньше 0). | Когда нужна **непрерывная шкала аномальности**. |
| **`decision_function(X)`** | Возвращает **разницу между `score_samples` и порогом `rho`** (0 - граница нормы). | Когда надо **точно определить, аномалия это или нет**. |

`score_samples` полезен, если небходимо **ранжировать аномалии**, а `decision_function`, если еще нужно разделить на **норма/аномалия**.

In [ ]:
data_test.loc[:,'svm_score'] = ocsvm.score_samples(X_test)
data_test.loc[:,'svm_decision_function'] = ocsvm.decision_function(X_test)

In [ ]:
fig = plt.figure();

signal = 'svm_score'

plt.scatter(
    data_test[data_test['anomaly'] == 1][signal].index,
    data_test[data_test['anomaly'] == 1][signal].values,
    s=70, facecolors='none', edgecolors='r',
    label= "Аномальные данные", alpha=0.5);

plt.scatter(
    data_test[data_test['anomaly_svm'] == 1][signal].index,
    data_test[data_test['anomaly_svm'] == 1][signal].values,
    s=10, facecolors='r', edgecolors='r',
    label= "Выявлена аномалия", alpha=0.5);

plt.scatter(
    data_test[data_test['anomaly'] == 0][signal].index,
    data_test[data_test['anomaly'] == 0][signal].values,
    s=70, facecolors='none', edgecolors='g',
    label= "Нормальные данные", alpha=0.5);

plt.scatter(
    data_test[data_test['anomaly_svm'] == 0][signal].index,
    data_test[data_test['anomaly_svm'] == 0][signal].values,
    s=10, facecolors='g', edgecolors='g',
    label= "Не выявлена аномалия", alpha=0.5);

# plt.plot(data_test.index,
#          np.ones(data_test.shape[0])*100, color='red', linewidth=5);

plt.ylabel(signal, fontsize=20);
plt.legend(prop={'size': 16}, loc='lower right');

# fig.savefig('fig10.png', dpi=fig.dpi, bbox_inches='tight');

In [ ]:
fig = plt.figure();

signal = 'svm_decision_function'

plt.scatter(
    data_test[data_test['anomaly'] == 1][signal].index,
    data_test[data_test['anomaly'] == 1][signal].values,
    s=70, facecolors='none', edgecolors='r',
    label= "Аномальные данные", alpha=0.5);

plt.scatter(
    data_test[data_test['anomaly_svm'] == 1][signal].index,
    data_test[data_test['anomaly_svm'] == 1][signal].values,
    s=10, facecolors='r', edgecolors='r',
    label= "Выявлена аномалия", alpha=0.5);

plt.scatter(
    data_test[data_test['anomaly'] == 0][signal].index,
    data_test[data_test['anomaly'] == 0][signal].values,
    s=70, facecolors='none', edgecolors='g',
    label= "Нормальные данные", alpha=0.5);

plt.scatter(
    data_test[data_test['anomaly_svm'] == 0][signal].index,
    data_test[data_test['anomaly_svm'] == 0][signal].values,
    s=10, facecolors='g', edgecolors='g',
    label= "Не выявлена аномалия", alpha=0.5);

# plt.plot(data_test.index,
#          np.ones(data_test.shape[0])*100, color='red', linewidth=5);

plt.ylabel(signal, fontsize=20);
plt.legend(prop={'size': 16}, loc='lower right');

# fig.savefig('fig10.png', dpi=fig.dpi, bbox_inches='tight');